In [16]:
!pip install geemap

In [17]:
!pip install earthengine-api
!pip install folium

In [18]:
!earthengine authenticate

Authenticate: Credentials already exist.  Use --force to refresh.


In [19]:
import ee
import geemap 
import folium
ee.Authenticate()


True

In [20]:
ee.Initialize()


In [21]:
# Define the image
dataset = ee.Image('ACA/reef_habitat/v2_0')

# Teti'aroa, an atoll in French Polynesia.
center_lon, center_lat, zoom = -149.56194, -17.00872, 13
map_center = [center_lat, center_lon]


The latitude, longitude, and zoom level are defined to center the map on Teti'aroa, an atoll in French Polynesia.

# Inspection on data

In [25]:
# Get basic information about the dataset
info = dataset.getInfo()

# Display some key metadata
print("Dataset ID:", info['id'])
print("Description:", info.get('description', 'No description available'))
print("Bands:", [band['id'] for band in info['bands']])
print("Properties:", info.get('properties', {}))


Dataset ID: ACA/reef_habitat/v2_0
Description: No description available
Bands: ['geomorphic', 'benthic', 'reef_mask']
Properties: {'benthic_class_palette': ['000000', 'ffffbe', 'e0d05e', 'b19c3a', '668438', 'ff6161', '9bcc4f'], 'reef_mask_class_names': ['Not reef', 'Reef'], 'geomorphic_class_values': [0, 11, 12, 13, 14, 15, 16, 21, 22, 23, 24, 25], 'benthic_class_names': ['Unmapped', 'Sand - Sand is any soft-bottom area dominated by fine unconsolidated sediments.', 'Rubble - Rubble is any habitat featuring loose, rough fragments of broken reef material.', 'Rock - Rock is any exposed area of hard bare substrate.', 'Seagrass - Seagrass is any habitat where seagrass is the dominant biota.', 'Coral/Algae - Coral/Algae is any hard-bottom area supporting living coral and/or algae.', 'Microalgal Mats - Microalgal Mats are any visible accumulations of microscopic algae in sandy sediments.\n'], 'reef_mask_class_values': [0, 1], 'reef_mask_class_palette': ['000000', 'ffffff'], 'system:footprint'

# Visualising the dataset

In [27]:
# Create a map to visualize the dataset
m = geemap.Map()

# Add the dataset to the map
m.addLayer(dataset, {}, 'Reef Habitat')

# Set a view to a specific region
m.set_center(lon=150.0, lat=-23.0, zoom=5)

# Display the map
m


Map(center=[-23.0, 150.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

# Examining Band Statistics

In [52]:
# Get band names
band_names = [band['id'] for band in info['bands']]

# Extract statistics for each band
band_stats = {}
for band in band_names:
    band_info = dataset.select(band).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=ee.Geometry.Polygon([[-180, -90], [180, -90], [180, 90], [-180, 90]]),
        scale=10000,
        bestEffort=True,
    ).getInfo()
    band_stats[band] = band_info

print("Band Statistics:", band_stats)


Band Statistics: {'geomorphic': {'geomorphic': 14.745318352059925}, 'benthic': {'benthic': 13.903083700440527}, 'reef_mask': {'reef_mask': 1}}


# Exporting to local for further Analysis

In [28]:
# Export the dataset to Google Drive as a GeoTIFF for further analysis
task = ee.batch.Export.image.toDrive(
    image=dataset,
    description='reef_habitat',
    fileNamePrefix='reef_habitat',
    scale=10000,
    region=ee.Geometry.Polygon([[-180, -90], [180, -90], [180, 90], [-180, 90]]),
    fileFormat='GeoTIFF'
)

# Start the export task
task.start()

# Check task status
task.status()


{'state': 'READY',
 'description': 'reef_habitat',
 'priority': 100,
 'creation_timestamp_ms': 1714105899970,
 'update_timestamp_ms': 1714105899970,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'IJTS3LJ2UCUNF2WF4DNNR6IZ',
 'name': 'projects/earthengine-legacy/operations/IJTS3LJ2UCUNF2WF4DNNR6IZ'}

# Reef, Geomorphic and Benthic

In [29]:
import folium  # For creating interactive maps
import ee  # For interacting with Google Earth Engine
import geemap  # For adding Earth Engine layers to Folium

# Create a Folium map object with the correct center and zoom level
center_lon, center_lat, zoom = -149.56194, -17.00872, 13  # Teti'aroa, French Polynesia
map_center = [center_lat, center_lon]  # Ensure coordinates are in the correct order
map = folium.Map(location=map_center, zoom_start=zoom)  # Creating the Folium map

# Define the dataset from Earth Engine
dataset = ee.Image('ACA/reef_habitat/v2_0')

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add this method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Add layers to the map with different visual parameters
# Reef extent
reef_extent_vis = {'min': 0, 'max': 1, 'palette': ['blue']}
map.add_ee_layer(dataset.select('reef_mask').selfMask(), reef_extent_vis, 'Global Reef Extent')

# Geomorphic zonation classification
geomorphic_vis = {
    'min': 1,
    'max': 8,
    'palette': ['FF5733', 'FFBD33', 'FFC300', 'FF6F61', 'FF3333', 'FF77B5', 'FF6F61', 'FF9B33']  # Warm colors
}
map.add_ee_layer(dataset.select('geomorphic').selfMask(), geomorphic_vis, 'Geomorphic Zonation')

# Benthic habitat classification
benthic_vis = {
    'min': 1,
    'max': 10,
    'palette': ['0000FF', '0000AA', '0066FF', '0099FF', '00CCFF', '00FFCC', '33FF99', '66FF99', '99FF99', 'CCFF99']  # Cool colors
}
map.add_ee_layer(dataset.select('benthic').selfMask(), benthic_vis, 'Benthic Habitat')

# Add layer control for toggling layers
folium.LayerControl().add_to(map)

# Display the map
map


In [30]:
import folium  # For creating interactive maps
import ee  # For interacting with Google Earth Engine
import geemap  # For adding Earth Engine layers to Folium

center_lon, center_lat, zoom = 78.9629, 20.5937, 5  # India
map_center = [center_lat, center_lon]  # Ensure coordinates are in the correct order
map = folium.Map(location=map_center, zoom_start=zoom)  # Creating the Folium map

# Define the dataset from Earth Engine
dataset = ee.Image('ACA/reef_habitat/v2_0')

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add this method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Add layers to the map with different visual parameters
# Reef extent
reef_extent_vis = {'min': 0, 'max': 1, 'palette': ['blue']}
map.add_ee_layer(dataset.select('reef_mask').selfMask(), reef_extent_vis, 'Global Reef Extent')

# Geomorphic zonation classification
geomorphic_vis = {
    'min': 1,
    'max': 8,
    'palette': ['FF5733', 'FFBD33', 'FFC300', 'FF6F61', 'FF3333', 'FF77B5', 'FF6F61', 'FF9B33']  # Warm colors
}
map.add_ee_layer(dataset.select('geomorphic').selfMask(), geomorphic_vis, 'Geomorphic Zonation')

# Benthic habitat classification
benthic_vis = {
    'min': 1,
    'max': 10,
    'palette': ['0000FF', '0000AA', '0066FF', '0099FF', '00CCFF', '00FFCC', '33FF99', '66FF99', '99FF99', 'CCFF99']  # Cool colors
}
map.add_ee_layer(dataset.select('benthic').selfMask(), benthic_vis, 'Benthic Habitat')

# Add layer control for toggling layers
folium.LayerControl().add_to(map)

# Display the map
map


# Choropleth

In [31]:
# Create a Folium map centered on French Polynesia
center_lon, center_lat = -149.56194, -17.00872  # Coordinates for French Polynesia
map = folium.Map(location=[center_lat, center_lon], zoom_start=11)  # Zoom level for closer view

# Select the Earth Engine dataset (ensure the band exists and has data)
reef_dataset = ee.Image('ACA/reef_habitat/v2_0')

# Select geomorphic band for choropleth
geomorphic_image = reef_dataset.select('geomorphic')  # Ensure this band exists and has data

# Visualization parameters for the choropleth
geomorphic_vis_params = {
    'min': 1,
    'max': 8,
    'palette': ['FF5733', 'FFBD33', 'FFC300', 'FF6F61', 'FF3333', 'FF77B5', 'FF6F61', 'FF9B33']  # Warm colors for zones
}

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add this method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Add the geomorphic layer to the Folium map
map.add_ee_layer(geomorphic_image.selfMask(), geomorphic_vis_params, 'Geomorphic Zones')

# Add layer control for toggling layers
folium.LayerControl().add_to(map)

# Display the map
map

# Loading Land Surface Temperature dataset

In [42]:
import folium  # For creating interactive maps
import ee  # For interacting with Google Earth Engine

# Authenticate and initialize Earth Engine (assuming you've already done this)
ee.Initialize()

# Create a simple Folium map centered on the equator
map = folium.Map(location=[0, 0], zoom_start=2)  # Create a basic map

# Access the Oxford LST Image Collection
oxford_lst_collection = ee.ImageCollection("Oxford/MAP/LST_Day_5km_Monthly")

# Get the latest image from the collection and print available bands
latest_lst_image = oxford_lst_collection.sort("system:time_start", False).first()
band_names = latest_lst_image.bandNames().getInfo()
print("Available bands:", band_names)

Available bands: ['Mean', 'FilledProportion']


In [44]:
lst_image = latest_lst_image.select('Mean')

# Visualization parameters for LST
lst_vis_params = {
    'min': 250,
    'max': 350,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr="Map data © Google Earth Engine",
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add this method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Add the Land Surface Temperature layer to the map using lst_vis_params
map.add_ee_layer(lst_image, lst_vis_params, 'Land Surface Temperature')

# Add a Layer Control for toggling layers
folium.LayerControl().add_to(map)

# Display the map
map

In [88]:
import ee  # Import Earth Engine
import folium  # Import Folium for mapping

# Authenticate and initialize Earth Engine
ee.Authenticate()  # Authenticate Earth Engine
ee.Initialize()  # Initialize Earth Engine

# Create a Folium map centered on French Polynesia
center_lat, center_lon = -17.00872, -149.56194  # Coordinates for French Polynesia
map = folium.Map(location=[center_lat, center_lon], zoom_start=11)  # Adjust the zoom level as needed

# Load the reef dataset
reef_dataset = ee.Image('ACA/reef_habitat/v2_0')
reef_dataset_band = reef_dataset.select("reef_mask")  # Replace with correct band name

# Load the Oxford LST dataset and get the correct band
oxford_lst_collection = ee.ImageCollection("Oxford/MAP/LST_Day_5km_Monthly")  # The Oxford LST collection
latest_lst = oxford_lst_collection.sort('system:time_start', False).first()  # Get the latest image in the collection

# Select the 'Mean' band for Land Surface Temperature
lst_image = latest_lst.select('Mean')  # Use 'Mean' as it represents temperature

# Visualization parameters for LST
lst_vis_params = {
    'min': 250,  # Minimum temperature (adjust based on dataset)
    'max': 350,  # Maximum temperature
    'palette': ['blue', 'cyan', 'green', 'yellow', 'orange', 'red'],  # Color gradient for temperature
}

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)  # Get map ID for the layer
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the method to Folium Map class
folium.Map.add_ee_layer = add_ee_layer  # Add method to Folium class

# Add the reef dataset to the Folium map
reef_vis_params = {
    'min': 0,  # Adjust as needed
    'max': 1,  # Adjust as needed
    'palette': ['blue'],  # Only for single-band visualization
}

map.add_ee_layer(reef_dataset_band, reef_vis_params, 'Reef Habitat')  # Add reef habitat to the map

# Add the LST dataset to the Folium map
map.add_ee_layer(lst_image, lst_vis_params, 'Land Surface Temperature')  # Add LST layer to the map

# Add layer control for toggling layers
folium.LayerControl().add_to(map)

# Display the map
map  # Return the map for visualization


In [45]:
center_lon, center_lat, zoom = -149.56194, -17.00872, 13  # Teti'aroa, French Polynesia
map_center = [center_lat, center_lon]  # Coordinates for French Polynesia
map = folium.Map(location=map_center, zoom_start=zoom)  # Creating the Folium map

# Define the dataset from Earth Engine
oxford_lst_collection = ee.ImageCollection("Oxford/MAP/LST_Day_5km_Monthly")

# Get the latest image from the collection
latest_lst = oxford_lst_collection.sort("system:time_start", False).first()

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr="Map data © Google Earth Engine",
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)

# Add this method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Define visualization parameters for Land Surface Temperature
lst_vis_params = {
    "min": 250,  # Minimum temperature
    "max": 350,  # Maximum temperature
    "palette": ["blue", "cyan", "green", "yellow", "orange", "red"],  # Temperature gradient
}

# Add the latest LST image to the Folium map
map.add_ee_layer(latest_lst.select("Mean"), lst_vis_params, "Land Surface Temperature")

# Add layer control for toggling layers
folium.LayerControl().add_to(map)

# Display the map
map  # This will display the map with the specified dataset


In [50]:
import folium  # For creating interactive maps
import ee  # For interacting with Google Earth Engine

# Authenticate and initialize Earth Engine
ee.Authenticate()  # Authenticate with Earth Engine
ee.Initialize()  # Initialize Earth Engine

# Create a simple Folium map centered on the equator
map = folium.Map(location=[0, 0], zoom_start=2)  # Create a basic map

# Define the Land Surface Temperature dataset from Earth Engine
oxford_lst_collection = ee.ImageCollection("Oxford/MAP/LST_Day_5km_Monthly")

# Get the latest image from the collection
latest_lst = oxford_lst_collection.sort("system:time_start", False).first()

# Define a simplified visualization with a broader range
lst_vis_params = {
    "min": 250,  # Lower minimum to cover more temperature range
    "max": 320,  # Maximum temperature 
    "palette": ["blue", "cyan", "green", "yellow", "orange", "red"],  # Temperature gradient
}

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)  # Get map ID
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr="Map data © Google Earth Engine",
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)

# Add this method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Add the Land Surface Temperature layer to the map
map.add_ee_layer(latest_lst.select("Mean"), lst_vis_params, "Land Surface Temperature")

# Add a Layer Control for toggling layers
folium.LayerControl().add_to(map)

# Display the map
map  # This should display the map with the latest LST data


# Sea Surface Tension dataset

In [51]:
sst_dataset = ee.ImageCollection("NOAA/CDR/SST_WHOI/V2")
first_image = sst_dataset.first()

# Get the band names
band_names = first_image.bandNames().getInfo()

print("Band Names:", band_names)

Band Names: ['sea_surface_temperature', 'fill_missing_qc']


In [52]:
import folium  # For creating interactive maps
import ee  # For interacting with Google Earth Engine
import geemap  # Optional, for additional visualization tools

# Authenticate and initialize Earth Engine
ee.Authenticate()  # Authenticate with Earth Engine
ee.Initialize()  # Initialize Earth Engine

# Create a Folium map centered on the equator for a global view
world_map = folium.Map(location=[0, 0], zoom_start=2)  # Zoom level set for global view

# Access the Sea Surface Temperature dataset from NOAA
sst_dataset = ee.ImageCollection("NOAA/CDR/SST_WHOI/V2")

# Get the latest image from the collection and select the 'sea_surface_temperature' band
latest_sst_image = sst_dataset.sort("system:time_start", False).first()  # Get the latest image
sst_image = latest_sst_image.select("sea_surface_temperature")  # Select the correct band

# Define visualization parameters for Sea Surface Temperature
sst_vis_params = {
    "min": 273,  # Minimum temperature in Kelvin
    "max": 303,  # Maximum temperature in Kelvin
    "palette": ["darkblue", "blue", "cyan", "green", "yellow", "orange", "red"],  # Gradient for SST
}

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)  # Get map ID for the layer
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr="Map data © Google Earth Engine",
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)

# Add this method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Add the Sea Surface Temperature layer to the map
world_map.add_ee_layer(sst_image, sst_vis_params, "Sea Surface Temperature")

# Add a Layer Control for toggling layers
folium.LayerControl().add_to(world_map)

# Display the map
world_map  # This should display the map with the Sea Surface Temperature data


In [54]:
import folium  # For creating interactive maps
import ee  # For interacting with Google Earth Engine

# Authenticate and initialize Earth Engine
ee.Authenticate()  # Authenticate with Earth Engine
ee.Initialize()  # Initialize Earth Engine

# Create a Folium map centered on the equator for a global view
map = folium.Map(location=[0, 0], zoom_start=2)  # Basic map with global view

# Load the Reef Mask as an Image, not an ImageCollection
reef_image = ee.Image("ACA/reef_habitat/v2_0")

# Load the Oxford LST ImageCollection
oxford_lst_collection = ee.ImageCollection("Oxford/MAP/LST_Day_5km_Monthly")

# Get the latest image from the collection for land surface temperature
latest_lst = oxford_lst_collection.sort("system:time_start", False).first()

# Load the Sea Surface Temperature dataset and get the latest image
sst_dataset = ee.ImageCollection("NOAA/CDR/SST_WHOI/V2").sort("system:time_start", False).first()

# Visualization parameters for Reef Mask
reef_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue'],  # Palette for visualizing reef mask
}

# Visualization parameters for Land Surface Temperature
lst_vis_params = {
    'min': 250,
    'max': 350,
    'palette': ["blue", "cyan", "green", "yellow", "orange", "red"],  # Temperature gradient
}

# Visualization parameters for Sea Surface Temperature
sst_vis_params = {
    'min': 273,
    'max': 303,
    'palette': ["darkblue", "blue", "cyan", "green", "yellow", "orange", "red"],  # Gradient for SST
}

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)  # Get map ID
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr="Map data © Google Earth Engine",
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)

# Add this method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Add the Reef Mask layer to the map
map.add_ee_layer(reef_image.select("reef_mask"), reef_vis_params, "Reef Mask")

# Add the Land Surface Temperature layer to the map
map.add_ee_layer(latest_lst.select("Mean"), lst_vis_params, "Land Surface Temperature")

# Add the Sea Surface Temperature layer to the map
map.add_ee_layer(sst_dataset.select("sea_surface_temperature"), sst_vis_params, "Sea Surface Temperature")

# Add a Layer Control for toggling layers
folium.LayerControl().add_to(map)

# Display the map
map  # This should display the map with all layers


In [55]:
import folium  # For creating interactive maps
import ee  # For interacting with Google Earth Engine

# Authenticate and initialize Earth Engine
ee.Authenticate()  # Authenticate with Earth Engine
ee.Initialize()  # Initialize Earth Engine

# Create a Folium map centered on the equator for a global view
map = folium.Map(location=[0, 0], zoom_start=2)  # Basic map with global view

# Load the Reef Mask as an Image
reef_image = ee.Image("ACA/reef_habitat/v2_0")

# Define visualization parameters for the Reef Mask
reef_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue'],  # Palette for visualizing reef mask
}

# Define visualization parameters for Land Surface Temperature (adjusted if necessary)
lst_vis_params = {
    'min': 250,  # Adjusted minimum temperature
    'max': 350,  # Adjusted maximum temperature
    'palette': ["blue", "cyan", "green", "yellow", "orange", "red"],  # Temperature gradient
}

# Define visualization parameters for Sea Surface Temperature (based on temperature range)
sst_vis_params = {
    "min": 270,  # Adjusted minimum temperature based on actual range
    "max": 310,  # Adjusted maximum temperature
    "palette": ["darkblue", "blue", "cyan", "green", "yellow", "orange", "red"],  # SST gradient
}

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)  # Get map ID
    folium.raster_layers.TileLayer(
        tiles=map_id_dict["tile_fetcher"].url_format,
        attr="Map data © Google Earth Engine",
        name=name,
        overlay=True,
        control=True,
    ).add_to(self)

# Add this method to Folium
folium.Map.add_ee_layer = add_ee_layer

# Add the Reef Mask layer to the map
map.add_ee_layer(reef_image.select("reef_mask"), reef_vis_params, "Reef Mask")

# Add the Land Surface Temperature layer to the map
oxford_lst = ee.ImageCollection("Oxford/MAP/LST_Day_5km_Monthly").sort("system:time_start", False).first()
map.add_ee_layer(oxford_lst.select("Mean"), lst_vis_params, "Land Surface Temperature")

# Add the Sea Surface Temperature layer to the map
sst_image = ee.ImageCollection("NOAA/CDR/SST_WHOI/V2").sort("system:time_start", False).first()
map.add_ee_layer(sst_image.select("sea_surface_temperature"), sst_vis_params, "Sea Surface Temperature")

# Add a Layer Control for toggling layers
folium.LayerControl().add_to(map)

# Display the map
map  # This should display the map with all layers


In [62]:
import geemap  # For creating interactive maps with Earth Engine
import ee  # For interacting with Google Earth Engine

# Authenticate and initialize Earth Engine
ee.Authenticate()  # Authenticate with Earth Engine
ee.Initialize()  # Initialize Earth Engine

# Create a Geemap map centered on the equator with a zoom level suitable for a global view
map = geemap.Map(center=[0, 0], zoom=2)  # Create a Geemap object for global visualization

# Load the Reef Mask as an Image
reef_image = ee.Image("ACA/reef_habitat/v2_0")

# Load the latest Land Surface Temperature from Oxford
oxford_lst = ee.ImageCollection("Oxford/MAP/LST_Day_5km_Monthly").sort("system:time_start", False).first()

# Load the latest Sea Surface Temperature from NOAA
sst_image = ee.ImageCollection("NOAA/CDR/SST_WHOI/V2").sort("system:time_start", False).first()

# Visualization parameters for the Reef Mask
reef_vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue', "darkblue"],  # A simple palette for reef mask visualization
}

# Visualization parameters for Land Surface Temperature
lst_vis_params = {
    'min': 250,
    'max': 350,
    'palette': ["yellow", "red"],  # A simple color for visualization without gradients
}

# Visualization parameters for Sea Surface Temperature
sst_vis_params = {
    'min': 270,
    'max': 310,
    'palette': ["green","darkgreen"],  # A single-color palette for SST visualization
}

# Add the Reef Mask layer to the map
map.addLayer(reef_image.select("reef_mask"), reef_vis_params, "Reef Mask")

# Add the Land Surface Temperature layer to the map
map.addLayer(oxford_lst.select("Mean"), lst_vis_params, "Land Surface Temperature")

# Add the Sea Surface Temperature layer to the map
map.addLayer(sst_image.select("sea_surface_temperature"), sst_vis_params, "Sea Surface Temperature")

# Add a Layer Control for toggling layers
map.addLayerControl()

# Display the map
map  # This should display the map with the three layers


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…